# Generic Abstract Agent Design

## Design Principles

### What's Common Across All Agents:
1. **State Management**: All agents need message history and memory persistence
2. **Graph Structure**: Basic pattern of reasoning node → tools → back to reasoning
3. **Public API**: `call()` and `resume()` methods
4. **Memory/Checkpointing**: State persistence across conversations

### What Varies Between Agents:
1. **Tools**: Different agents need different capabilities
2. **LLM Configuration**: Model, temperature, system prompts
3. **State Schema**: May need additional fields beyond messages
4. **Custom Nodes**: Some agents may need preprocessing, validation, etc.
5. **Routing Logic**: May need custom conditional edges

### Design Pattern: Template Method + Hooks

We'll use:
- **Abstract Base Class**: Defines the skeleton of the algorithm
- **Template Methods**: `_build_graph()` orchestrates the building process
- **Hook Methods**: Subclasses override to customize behavior
- **Default Implementations**: Common patterns provided out-of-the-box

## Benefits of BaseAgent Design

### 1. **Minimal Implementation Required**
- Subclasses only need to implement `get_tools()`
- All common functionality is inherited

### 2. **Flexible Customization**
- Override hooks only where needed
- Default implementations handle common cases
- Can customize LLM, state, nodes, routing independently

### 3. **Consistent API**
- All agents have the same `call()` and `resume()` interface
- Easy to swap agents in your code
- Predictable behavior across agent types

### 4. **Reusable Patterns**
- Template method pattern ensures consistent graph building
- Hooks allow customization without duplicating code
- Easy to add new agent types

### 5. **Maintainability**
- Common logic in one place (BaseAgent)
- Changes to base behavior benefit all agents
- Clear separation of concerns



In [1]:
from dotenv import load_dotenv
from typing import Dict, List, Any, Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langsmith import traceable
from langgraph.types import interrupt, Command

from uuid import uuid4
load_dotenv()


True

In [2]:
# llm.invoke("Hello, world!")
# Tools
@tool
def convert_currency(currency: str, amount_in_usd: float) -> str:
    """
    Convert an amount from USD to the specified currency.

    Supported currencies and their hardcoded exchange rates:
    - USD: 1 USD = 1 USD
    - EUR: 1 USD = 0.85 EUR
    - ARS: 1 USD = 1430 ARS

    Args:
        currency (str): The currency code to convert to ("USD", "EUR", or "ARS").
        amount_in_usd (float): The amount in USD to convert.

    Returns:
        str: The converted amount as a string (in the destination currency).

    Raises:
        ValueError: If the currency is not supported.
    """
    if currency == "USD":
        return amount_in_usd
    elif currency == "EUR":
        return amount_in_usd * 0.85
    elif currency == "ARS":
        return amount_in_usd * 1430
    else:
        raise ValueError(f"Currency {currency} not supported")


@tool
def buy_usd(amount: float) -> str:
    """
    Buy USD.
    """
    print(f"[TOOL] buy_usd called with amount: {amount} USD")
    
    decision = interrupt(
        f"Buy {amount} USD"
    )
    
    print(f"[TOOL] Human decision received: {decision}")
    
    if decision == "yes":
        print(f"[TOOL] ✅ Human approved! Executing purchase of {amount} USD...")
        result = f"Buying {amount} USD"
        print(f"[TOOL] ✅ Purchase completed: {result}")
        return result
    else:
        print(f"[TOOL] ❌ Human rejected the purchase of {amount} USD")
        return "Buying rejected"


tools = [convert_currency, buy_usd]

llm = ChatOpenAI(model="gpt-5-nano", temperature=0.7)
llm_with_tools = llm.bind_tools(tools)



In [3]:
# BaseAgent - Generic Abstract Agent Class

from abc import ABC, abstractmethod
from langchain_core.language_models import BaseChatModel

class BaseAgent(ABC):
    """
    Abstract base class for creating LangGraph-based agents.
    
    This class provides a template for building agents with:
    - LLM reasoning capabilities
    - Tool execution
    - State management and memory persistence
    - Human-in-the-loop support
    
    Subclasses must implement:
    - `get_tools()`: Return list of tools for this agent
    - Optionally override hooks for customization
    """
    
    def __init__(
        self,
        llm: BaseChatModel = None,
        checkpointer = None,
        **kwargs
    ):
        """
        Initialize the agent.
        
        Args:
            llm: Chat model instance. If None, uses `create_llm()` hook.
            checkpointer: Checkpointer for state persistence. If None, uses MemorySaver.
            **kwargs: Additional configuration passed to hooks
        """
        # Initialize LLM
        if llm is None:
            self.llm = self.create_llm(**kwargs)
        else:
            self.llm = llm
        
        # Get tools (subclass must implement)
        self.tools = self.get_tools(**kwargs)
        
        # Bind tools to LLM
        self.llm_with_tools = self.llm.bind_tools(self.tools)
        
        # Initialize checkpointer
        if checkpointer is None:
            self.checkpointer = self.create_checkpointer(**kwargs)
        else:
            self.checkpointer = checkpointer
        
        # Build and compile the graph
        self._build_graph(**kwargs)
    
    # ========== Abstract Methods (Must Implement) ==========
    
    @abstractmethod
    def get_tools(self, **kwargs) -> List:
        """
        Return the list of tools for this agent.
        
        Args:
            **kwargs: Configuration passed from __init__
        
        Returns:
            List of tool instances
        """
        pass
    
    # ========== Hook Methods (Can Override) ==========
    
    def create_llm(self, **kwargs) -> BaseChatModel:
        """
        Create the LLM instance. Override to customize.
        
        Default: Creates ChatOpenAI with gpt-5-nano
        
        Args:
            **kwargs: Configuration passed from __init__
        
        Returns:
            BaseChatModel instance
        """
        return ChatOpenAI(model="gpt-5-nano", temperature=0.7)
    
    def create_checkpointer(self, **kwargs):
        """
        Create the checkpointer for state persistence. Override to customize.
        
        Default: Creates MemorySaver
        
        Args:
            **kwargs: Configuration passed from __init__
        
        Returns:
            Checkpointer instance
        """
        return MemorySaver()
    
    def get_state_schema(self, **kwargs) -> TypedDict:
        """
        Define the state schema. Override to add custom fields.
        
        Default: Basic state with messages only
        
        Args:
            **kwargs: Configuration passed from __init__
        
        Returns:
            TypedDict class defining the state schema
        """
        class AgentState(TypedDict):
            messages: Annotated[List, add_messages]
        
        return AgentState
    
    def create_chatbot_node(self, state_schema: TypedDict, **kwargs):
        """
        Create the chatbot/reasoning node. Override for custom reasoning logic.
        
        Default: Standard LLM invocation with full message history
        
        Args:
            state_schema: The state TypedDict class
            **kwargs: Configuration passed from __init__
        
        Returns:
            Function that takes state and returns state updates
        """
        def chatbot(state: state_schema) -> Dict:
            """Reasoning node that invokes the LLM."""
            response = self.llm_with_tools.invoke(state["messages"])
            return {"messages": [response]}
        
        return chatbot
    
    def create_tools_node(self, **kwargs):
        """
        Create the tools execution node. Override for custom tool handling.
        
        Default: Standard ToolNode
        
        Args:
            **kwargs: Configuration passed from __init__
        
        Returns:
            ToolNode instance
        """
        return ToolNode(self.tools)
    
    def add_custom_nodes(self, builder: StateGraph, state_schema: TypedDict, **kwargs):
        """
        Add custom nodes to the graph. Override to add preprocessing, validation, etc.
        
        Default: No custom nodes
        
        Args:
            builder: The StateGraph builder
            state_schema: The state TypedDict class
            **kwargs: Configuration passed from __init__
        """
        pass
    
    def customize_routing(self, builder: StateGraph, **kwargs):
        """
        Customize routing logic. Override for custom conditional edges.
        
        Default: Standard tools_condition routing
        
        Args:
            builder: The StateGraph builder
            **kwargs: Configuration passed from __init__
        """
        # Default: Route chatbot output based on tool calls
        builder.add_conditional_edges(
            "chatbot",
            tools_condition
        )
    
    def customize_edges(self, builder: StateGraph, **kwargs):
        """
        Customize graph edges. Override for custom edge logic.
        
        Default: Tools loop back to chatbot
        
        Args:
            builder: The StateGraph builder
            **kwargs: Configuration passed from __init__
        """
        # Default: Tools loop back to chatbot for continued reasoning
        builder.add_edge("tools", "chatbot")
    
    # ========== Template Method (Orchestrates Building) ==========
    
    def _build_graph(self, **kwargs):
        """
        Build and compile the agent graph using LangChain's create_agent.
        
        Uses create_agent internally for production-ready implementation.
        For advanced customization (custom routing/edges/nodes), override this method
        or use _build_graph_manual().
        """
        # Check if user wants manual graph building
        if kwargs.get("use_manual_graph", False):
            self._build_graph_manual(**kwargs)
            return
        
        # Try to use create_agent for standard cases
        try:
            self._build_graph_with_create_agent(**kwargs)
        except Exception:
            # Fall back to manual building if create_agent fails
            # (e.g., due to unsupported customization)
            self._build_graph_manual(**kwargs)
    
    def get_prompt(self, **kwargs):
        """
        Get the system prompt for the agent. Override to customize.
        
        Returns:
            str, SystemMessage, or None for default behavior
        """
        return None
    
    def get_pre_model_hook(self, **kwargs):
        """
        Get a pre-model hook for preprocessing. Override to customize.
        
        Returns:
            Callable or None for no preprocessing
        """
        return None
    
    def get_post_model_hook(self, **kwargs):
        """
        Get a post-model hook for post-processing. Override to customize.
        
        Returns:
            Callable or None for no post-processing
        """
        return None
    
    def _build_graph_with_create_agent(self, **kwargs):
        """
        Build graph using LangChain's create_agent (non-deprecated).
        This provides a production-ready, tested implementation.
        """
        # Get state schema
        StateSchema = self.get_state_schema(**kwargs)
        
        # Get prompt, hooks from hook methods
        prompt = self.get_prompt(**kwargs)
        pre_model_hook = self.get_pre_model_hook(**kwargs)
        post_model_hook = self.get_post_model_hook(**kwargs)
        
        # Filter out our internal kwargs
        create_agent_kwargs = {k: v for k, v in kwargs.items() 
                              if k not in ['use_manual_graph']}
        
        # Use create_agent to build the graph (non-deprecated)
        self.graph = create_agent(
            model=self.llm_with_tools,
            tools=self.tools,
            prompt=prompt,
            state_schema=StateSchema,
            checkpointer=self.checkpointer,
            pre_model_hook=pre_model_hook,
            post_model_hook=post_model_hook,
            **create_agent_kwargs
        )
    
    def _build_graph_manual(self, **kwargs):
        """
        Build graph manually for advanced customization cases.
        This preserves full flexibility for custom routing, edges, and nodes.
        """
        # Get state schema
        StateSchema = self.get_state_schema(**kwargs)
        
        # Create nodes
        chatbot_node = self.create_chatbot_node(StateSchema, **kwargs)
        tools_node = self.create_tools_node(**kwargs)
        
        # Build graph
        builder = StateGraph(StateSchema)
        builder.add_node("chatbot", chatbot_node)
        builder.add_node("tools", tools_node)
        
        # Add custom nodes (hook)
        self.add_custom_nodes(builder, StateSchema, **kwargs)
        
        # Add edges
        builder.add_edge(START, "chatbot")
        
        # Customize routing (hook)
        self.customize_routing(builder, **kwargs)
        
        # Customize edges (hook)
        self.customize_edges(builder, **kwargs)
        
        # Compile graph with checkpointer
        self.graph = builder.compile(checkpointer=self.checkpointer)
    
    # ========== Public API ==========
    
    @traceable(run_type="llm", name="base_agent_call")
    def call(self, msg: str, config: Dict) -> Dict:
        """
        Send a message to the agent and get a response.
        
        Args:
            msg: The user message to send
            config: Configuration dict with thread_id: {"configurable": {"thread_id": "..."}}
        
        Returns:
            Dict containing the agent's state after processing the message
        """
        return self.graph.invoke({"messages": [HumanMessage(content=msg)]}, config=config)
    
    @traceable(run_type="llm", name="base_agent_resume")
    def resume(self, decision: str, config: Dict) -> Dict:
        """
        Resume an interrupted agent execution with a human decision.
        
        Args:
            decision: The human's decision (e.g., "yes" or "no")
            config: Configuration dict with thread_id: {"configurable": {"thread_id": "..."}}
        
        Returns:
            Dict containing the agent's state after resuming
        """
        print(f"[AGENT] Resuming with human decision: {decision}")
        result = self.graph.invoke(Command(resume=decision), config=config)
        print(f"[AGENT] Resume completed")
        return result
    
    def get_graph_image(self):
        """Display the agent's graph structure."""
        from IPython.display import Image, display
        display(Image(self.graph.get_graph().draw_mermaid_png()))


In [4]:
# ========== Examples: Different Agent Types Using BaseAgent ==========

# Example 1: Simple CurrencyAgent (minimal implementation - just override get_tools)
class CurrencyAgentV2(BaseAgent):
    """
    Currency conversion and USD purchasing agent.
    
    Simple example showing minimal implementation - just override get_tools().
    """
    
    def get_tools(self, **kwargs) -> List:
        """Return currency-related tools."""
        return [convert_currency, buy_usd]

# Example 2: Custom LLM configuration
class CustomLLMCurrencyAgent(BaseAgent):
    """Currency agent with custom LLM settings."""
    
    def get_tools(self, **kwargs):
        return [convert_currency, buy_usd]
    
    def create_llm(self, **kwargs):
        return ChatOpenAI(model="gpt-5-nano", temperature=0.3)  # Lower temperature

# Example 3: Agent with custom state schema
class AgentWithCustomState(BaseAgent):
    """Example showing how to add custom state fields."""
    
    def get_tools(self, **kwargs):
        return [convert_currency]  # Simplified tools
    
    def get_state_schema(self, **kwargs):
        """Add custom fields to state."""
        class CustomState(TypedDict):
            messages: Annotated[List, add_messages]
            user_balance: float  # Custom field
            transaction_count: int  # Custom field
        
        return CustomState

# Example 4: Agent with custom reasoning node
class AgentWithCustomReasoning(BaseAgent):
    """Example showing custom reasoning logic."""
    
    def get_tools(self, **kwargs):
        return [convert_currency]
    
    def create_chatbot_node(self, state_schema, **kwargs):
        """Custom reasoning with system prompt."""
        def chatbot(state: state_schema) -> Dict:
            # Add system message for context
            from langchain_core.messages import SystemMessage
            messages_with_system = [
                SystemMessage(content="You are a helpful currency conversion assistant."),
                *state["messages"]
            ]
            response = self.llm_with_tools.invoke(messages_with_system)
            return {"messages": [response]}
        
        return chatbot

# Example 5: Agent with preprocessing node
class AgentWithPreprocessing(BaseAgent):
    """Example showing how to add custom nodes."""
    
    def get_tools(self, **kwargs):
        return [convert_currency]
    
    def add_custom_nodes(self, builder, state_schema, **kwargs):
        """Add a preprocessing node."""
        def preprocess(state: state_schema) -> Dict:
            """Preprocess messages before reasoning."""
            # Example: Log or transform messages
            print(f"[PREPROCESS] Processing {len(state['messages'])} messages")
            return {}  # No state changes, just side effects
        
        builder.add_node("preprocess", preprocess)
    
    def customize_edges(self, builder, **kwargs):
        """Route through preprocessing first."""
        builder.add_edge(START, "preprocess")
        builder.add_edge("preprocess", "chatbot")
        builder.add_edge("tools", "chatbot")  # Still loop back from tools


In [5]:
# Example: Using CurrencyAgentV2 (same functionality as CurrencyAgent, but using BaseAgent)

# Create an instance
agent_v2 = CurrencyAgentV2()

# Use it exactly like the original CurrencyAgent
thread_id = str(uuid4())
config = {"configurable": {"thread_id": thread_id}}

# Test it
state = agent_v2.call("I have 100 EUR. How many USD are?", config)
print("Agent V2 response:", state['messages'][-1].content)

# Optional: View the graph structure
# agent_v2.get_graph_image()


/Users/maxicorona/Projects/Britecore/1. Drafts/agentic-poc/.venv/lib/python3.13/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


Agent V2 response: Using the given rate (1 USD = 0.85 EUR):

- 1 EUR = 1 / 0.85 ≈ 1.1764706 USD
- 100 EUR ≈ 100 × 1.1764706 ≈ 117.65 USD

So, 100 EUR is about 117.65 USD (based on the provided rate). If you want a different decimal precision or a live rate, tell me.
